In [2]:
from IPython.display import display
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from astropy.time import Time
import ephem
import astropy.units as units
from astropy.coordinates import SkyCoord
from tqdm import tqdm
import copy
import pickle
import matplotlib.cm as cm

/tmp/ipykernel_197305/517434333.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [3]:
from alerce.core import Alerce
client = Alerce()

In [4]:
def plotLC(oid, SN_det, SN_nondet):
    
    colors = {1: '#56E03A', 2: '#D42F4B'} 
    fig, ax = plt.subplots(figsize = (14, 7))
    fig.set_facecolor('white')
    labels = {1: 'g', 2: 'r'}
    markers = {1: 'o', 2: 's'}
    sizes = {1: 30, 2: 60}
    
    # loop the passbands
    for fid in [1, 2]:
        
        # plot detections if available
        mask = SN_det.fid == fid
        if np.sum(mask) > 0:
            # note that the detections index is candid and that we are plotting the psf corrected magnitudes
            ax.errorbar(SN_det[mask].mjd, SN_det[mask].magpsf, 
                yerr = SN_det[mask].sigmapsf, c=colors[fid], label=labels[fid], marker=markers[fid])
        
        # plot non detections if available
        if len(SN_nondet) != 0:
            mask = (SN_nondet.fid == fid) & (SN_nondet.diffmaglim > -900)
            if np.sum(mask) > 0:     
                # non detections index is mjd
                ax.scatter(SN_nondet[mask].mjd, SN_nondet[mask].diffmaglim, c=colors[fid], alpha = 0.5,
                    marker='v', label="lim.mag. %s" % labels[fid], s=sizes[fid])

    ax.set_title(oid)
    ax.set_xlabel("MJD")
    ax.set_ylabel("Apparent magnitude")
    ax.legend()
    ax.set_ylim(ax.get_ylim()[::-1])

In [5]:
def getObjectData(oid, doLC=False, dostamp=False):

    results = {"oid": oid}
        
    # query detections
    SN_det = client.query_detections(oid, format='pandas')
    SN_det = SN_det.sort_values("mjd")
    results["lc_det"] = SN_det
        
    # query non detections
    SN_nondet = client.query_non_detections(oid, format='pandas')
    if len(SN_nondet)!=0:
        SN_nondet = SN_nondet.sort_values("mjd")
        results["lc_nondet"] = SN_nondet
    
    # plot the LC
    if doLC:
        plotLC(oid, SN_det, SN_nondet)
        
    # show the first image stamp
    if dostamp:
        candid = results["lc_det"].loc[results["lc_det"].has_stamp].candid.min()
        stamps = client.get_stamps(oid, candid)
        science, ref, difference = stamps[0].data, stamps[1].data, stamps[2].data
        fig, ax = plt.subplots(ncols=3, figsize=(12, 6))
        fig.set_facecolor('white')
        for idx, im in enumerate([np.arcsinh(science), np.arcsinh(ref), difference]):
            ax[idx].imshow(im, cmap='viridis') # Log scale for visualization
            ax[idx].axes.get_xaxis().set_visible(False)
            ax[idx].axes.get_yaxis().set_visible(False)
        ax[0].set_title("oid: %s, candid: %s (science, reference and difference)" % (oid, candid), loc='left')
        fig.subplots_adjust(wspace = 0, hspace = 0)
        
    # return data
    return results

In [14]:
def get_objects_per_class(classearly="SN", pclassearly=0.8, n_objects=100):

    min_firstmjd = Time("2024-11-23T00:00:00", format="isot", scale="utc").mjd

    objects = client.query_objects(classifier="stamp_classifier",
                                   class_name=classearly,
                                   #classifier_version="stamp_classifier_1.0.4",
                                   probability=pclassearly,
                                   ranking=1,
                                   #ndet=[1, 50],
                                   #order_by="probability",
                                   #order_mode="DESC",
                                   count=False,
                                   first_mjd=[min_firstmjd, None],
                                   page_size=n_objects, 
                                   format='pandas')
    print(objects.shape)
    objects.head()
    objects.set_index("oid", inplace=True)
    objects.sort_values(by="ndet", inplace=True, ascending=False)
    return objects

In [22]:
n_objects = 200 # Objects per class to query
early_classes = ["AGN", "SN", "VS", "asteroid", "bogus"] # Class identifiers to query objects
objects = {} # Initialize dictionary to use the results per class
min_nobs = 1
for i, cl in enumerate(early_classes):
    print(cl)
    objects[cl] = get_objects_per_class(classearly=cl, n_objects=n_objects, pclassearly = 0.85)
    if i==0:
        print("Result of a query using the ALeRCE client")
        display(objects[cl].head())
        print("Columns available", objects[cl].columns)

AGN
(200, 23)
Result of a query using the ALeRCE client


,ndethist,ncovhist,mjdstarthist,mjdendhist,corrected,stellar,ndet,g_r_max,g_r_max_corr,g_r_mean,...,lastmjd,deltajd,meanra,meandec,sigmara,sigmadec,class,classifier,probability,step_id_corr
oid,,,,,,,,,,,,,,,,,,,,,
ZTF18aceutkr,1040,2501,58315.419317,60639.208669,True,False,632,NaN,NaN,NaN,...,60639.208669,1966.746563,355.482712,-6.341738,0.030544,0.030357,AGN,stamp_classifier,0.852734,24.5.2a6
ZTF19abdzdek,2218,3832,58336.285324,60639.185486,True,False,631,NaN,NaN,NaN,...,60639.185486,1964.719444,332.049971,26.476126,0.031938,0.028588,AGN,stamp_classifier,0.855873,24.5.2a6
ZTF19abniciz,865,2307,58390.276875,60623.192986,True,False,599,NaN,NaN,NaN,...,60623.192986,1929.716053,19.892743,-6.218994,0.032692,0.032500,AGN,stamp_classifier,0.858313,24.5.2a6
ZTF19aboutug,709,1507,58346.350567,60639.204433,True,False,546,NaN,NaN,NaN,...,60639.204433,1934.825949,350.178213,17.495614,0.054138,0.051633,AGN,stamp_classifier,0.850651,24.5.2a6
ZTF18aaszojl,972,3211,58487.500000,60078.250000,True,False,433,0.252649,0.746453,0.190323,...,60078.371539,1495.021053,229.980238,28.182625,0.000055,0.000051,AGN,stamp_classifier,0.850084,1.2.0


Columns available Index(['ndethist', 'ncovhist', 'mjdstarthist', 'mjdendhist', 'corrected',
       'stellar', 'ndet', 'g_r_max', 'g_r_max_corr', 'g_r_mean',
       'g_r_mean_corr', 'firstmjd', 'lastmjd', 'deltajd', 'meanra', 'meandec',
       'sigmara', 'sigmadec', 'class', 'classifier', 'probability',
       'step_id_corr'],
      dtype='object')
SN
(200, 23)
VS
(200, 23)
asteroid
(200, 23)
bogus
(200, 23)


In [15]:
objects["SN"]

,ndethist,ncovhist,mjdstarthist,mjdendhist,corrected,stellar,ndet,g_r_max,g_r_max_corr,g_r_mean,...,lastmjd,deltajd,meanra,meandec,sigmara,sigmadec,class,classifier,probability,step_id_corr
oid,,,,,,,,,,,,,,,,,,,,,
ZTF18aarkcxh,1885,2996,58208.513264,60639.160521,True,False,1009,NaN,NaN,NaN,...,60639.160521,2393.707569,291.159328,59.696227,0.102328,0.051633,SN,stamp_classifier,0.852988,24.5.2a6
ZTF18adkglwh,1132,2653,58218.469352,60495.464676,True,False,645,NaN,NaN,NaN,...,60495.462789,873.917593,288.057343,10.825110,0.004338,0.004261,SN,stamp_classifier,0.800888,24.5.2a6
ZTF17aaaexdy,687,1288,58101.184213,60639.277917,True,False,525,NaN,NaN,NaN,...,60639.277917,2281.870764,24.081110,66.619682,0.086512,0.034331,SN,stamp_classifier,0.876114,24.5.2a6
ZTF22aaoxume,512,4084,59467.196944,60636.093611,True,False,486,NaN,NaN,NaN,...,60636.093611,910.666065,317.433366,33.516861,0.029117,0.024275,SN,stamp_classifier,0.846718,24.5.2a6
ZTF22aadesap,832,3276,59670.382998,60576.179699,True,False,372,NaN,NaN,NaN,...,60576.179699,902.834977,232.765425,53.405337,0.042738,0.025478,SN,stamp_classifier,0.877757,24.5.2a6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZTF22aamrqfy,1,1419,59733.500000,59733.500000,True,False,1,NaN,NaN,NaN,...,59733.436586,0.000000,323.280358,34.936978,NaN,NaN,SN,stamp_classifier,0.860072,1.1.2
ZTF18adjdphn,6,2394,58232.500000,59733.250000,True,False,1,NaN,NaN,NaN,...,59733.342627,0.000000,285.657967,32.216255,NaN,NaN,SN,stamp_classifier,0.869561,1.1.2
ZTF21abyakwb,3,3155,59458.163322,59462.216146,True,False,1,NaN,NaN,NaN,...,59462.216146,0.000000,264.253458,38.160719,NaN,NaN,SN,stamp_classifier,0.873373,correction_1.0.6
